# Project: ReID

## Introduction

We wanted to reidentify a person, given some samples of the dataset. The idea is to learn the main features of the person in order to distinguish him/her from other people.



The first thing that we did was to import libraries.

In [ ]:
from sklearn.svm import SVC
from sklearn.svm import SVR, LinearSVR

from load_data import DataLoader
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, mean_absolute_error, mean_squared_error, roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression, LinearRegression, Lasso
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, Normalizer
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.base import BaseEstimator, ClassifierMixin, RegressorMixin
import numpy as np
import tqdm
import config
import math
import os
import matplotlib.pyplot as plt
import scikitplot as skplt
from utils import filter_and_split_dataset, filter_faces
from sklearn.metrics import DetCurveDisplay, RocCurveDisplay, ConfusionMatrixDisplay, roc_auc_score, roc_curve, det_curve, confusion_matrix, average_precision_score
import re




In [ ]:
d = DataLoader()
d.read_dataset()
d.shuffle_videos()
d.display_dataset()

The next thing to do was to design the learning setting. The first and easiest idea was to test a variety of classifiers, and see which works better for our features.

We tested different classifiers:

In [ ]:
def test_model(name, model_skel, model_clothes, model_face, X_skel, X_clothes, X_face, X_skel_test, X_clothes_test, X_face_test, y_train, y_test, y_face, y_face_test):
    model_skel.fit(X_skel, y_train)
    model_clothes.fit(X_clothes, y_train)
    model_face.fit(X_face, y_face)


    print("-" * 10)
    print("|", name, "|")
    # Test
    print("-" * 10)
    skel_pred = model_skel.predict(X_skel_test)
    print(classification_report(y_test, skel_pred))
    print("-" * 10)
    clothes_pred = model_clothes.predict(X_clothes_test)
    print(classification_report(y_test, clothes_pred))
    print("-" * 10)
    face_pred = model_face.predict(X_face_test)
    print(classification_report(y_face_test, face_pred))

    skel_pred_roc = model_skel.predict_proba(X_skel_test)
    clothes_pred_roc = model_clothes.predict_proba(X_clothes_test)
    face_pred_roc = model_face.predict_proba(X_face_test)

In [ ]:
X = np.load(config.SAMPLED_PATH + "video_1_sample_0_X.npy")
y = np.load(config.SAMPLED_PATH + "video_1_sample_0_y.npy")
X_skel, X_clothes, X_face, _, ___, ____, y_train, _____, y_face, ______ = filter_and_split_dataset(X, y)

X_2 = np.load(config.SAMPLED_PATH + "video_1_sample_1_X.npy")
y_2 = np.load(config.SAMPLED_PATH + "video_1_sample_1_y.npy")
X_skel_test, X_clothes_test, X_face_test, _, __, ___, y_test, ____, y_face_test, _____ = filter_and_split_dataset(X_2, y_2)



### Feature Importance


In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(20, 5))

# Skeleton
skel_feature = np.asarray(["chest", "right_upper_arm", "right_lower_arm", "left_upper_arm", "left_lower_arm", "right_upper_leg", "right_lower_leg", "left_upper_leg", "left_lower_leg"])
rf = RandomForestClassifier(criterion ='entropy', max_depth = None, max_features ='sqrt', min_samples_leaf = 1, min_samples_split = 2, n_estimators = 200, random_state = 42)
rf.fit(X_skel, y_train)
rf.feature_importances_

sorted_idx = rf.feature_importances_.argsort()
ax[0].barh(skel_feature[sorted_idx], rf.feature_importances_[sorted_idx])
# Clothes
clothes_feature = np.asarray([str(i) for i in range(0, 40)])
rf = RandomForestClassifier(criterion ='entropy', max_depth = None, max_features ='sqrt', min_samples_leaf = 1, min_samples_split = 10, n_estimators = 100, random_state = 4)
rf.fit(X_clothes, y_train)
rf.feature_importances_

sorted_idx = rf.feature_importances_.argsort()
ax[1].barh(clothes_feature[sorted_idx], rf.feature_importances_[sorted_idx])

# Face
face = np.asarray(["eyes_lenght", "eyes_width", "nose_heigth", "nose_width", "mouth_width", "mouth_height", "chin", "face_width", "face_height", "distance_nose_mouth", "distance_mouth_chin", "distance_between_eyes"])
rf = RandomForestClassifier(criterion ='entropy', max_depth = None, max_features ='sqrt', min_samples_leaf = 1, min_samples_split = 2, n_estimators = 200, random_state = 42)
rf.fit(X_face, y_face)
rf.feature_importances_

sorted_idx = rf.feature_importances_.argsort()
ax[2].barh(face[sorted_idx], rf.feature_importances_[sorted_idx])

fig.tight_layout()




In [ ]:

classifiers_skel = {
    "DecisionTreeClassifier": DecisionTreeClassifier(criterion='gini', max_depth=None, max_features='sqrt', min_samples_leaf=2, min_samples_split=10, random_state=2, splitter='random'),
    "RandomForestClassifier": RandomForestClassifier(criterion ='entropy', max_depth = None, max_features ='sqrt', min_samples_leaf = 1, min_samples_split = 2, n_estimators = 200, random_state = 42),
    "KNeighborsClassifier": KNeighborsClassifier(algorithm='auto', leaf_size=10, n_neighbors=3, p=1, weights='distance'),
    "LogisticRegression": LogisticRegression(solver="liblinear", max_iter=10000),
    "GaussianNB": GaussianNB(var_smoothing=1e-09),
    # "SVC": SVC(C = 1, kernel = "linear", degree = 3, gamma = "auto", probability = True, tol = 0.001, random_state = 42),
}

classifiers_clothes = {
    "DecisionTreeClassifier": DecisionTreeClassifier(criterion='gini', max_depth=None, max_features='sqrt', min_samples_leaf=1, min_samples_split=10, random_state=2, splitter='best'),
    "RandomForestClassifier": RandomForestClassifier(criterion ='entropy', max_depth = None, max_features ='sqrt', min_samples_leaf = 1, min_samples_split = 10, n_estimators = 100, random_state = 4),
    "KNeighborsClassifier": KNeighborsClassifier(algorithm='auto', leaf_size=10, n_neighbors=7, p=1, weights='distance'),
    "LogisticRegression": LogisticRegression(solver="liblinear", max_iter=10000),
    "GaussianNB": GaussianNB(var_smoothing=1e-09),
    # "SVC": SVC(C = 1, kernel = "linear", degree = 3, gamma = "auto", probability = True, tol = 0.001, random_state = 42),
}

classifiers_face = {
    "DecisionTreeClassifier": DecisionTreeClassifier(criterion='gini', max_depth=None, max_features='sqrt', min_samples_leaf=2, min_samples_split=5, random_state=2, splitter='random'),
    "RandomForestClassifier": RandomForestClassifier(criterion ='entropy', max_depth = None, max_features ='sqrt', min_samples_leaf = 1, min_samples_split = 2, n_estimators = 50, random_state = 42),
    "KNeighborsClassifier": KNeighborsClassifier(algorithm='auto', leaf_size=10, n_neighbors=9, p=1, weights='distance'),
    "LogisticRegression": LogisticRegression(solver="liblinear", max_iter=10000),
    "GaussianNB": GaussianNB(var_smoothing=1e-09),
    # "SVC": SVC(C = 1, kernel = "linear", degree = 3, gamma = "auto", probability = True, tol = 0.001, random_state = 42),
}

fig, ((ax_roc_skel, ax_roc_clothes, ax_roc_face), (ax_det_skel, ax_det_clothes, ax_det_face)) = plt.subplots(2, 3, figsize =(20, 10))

files = [f for f in os.listdir("./sampled") if re.match("(video_)([1-9]*)(_sample_)([0-9]*)_([Xy])(.npy)", f)]
files.sort()
samples = [(files[i], files[i+1], files[i+2], files[i+3]) for i in range(0, len(files) - 2, 4)]

def fit_dataset(model, X_train, y_train, X_test, y_test):    

    model.fit(X_train, y_train)

    res = model.predict_proba(X_test)[:, 1]

    fpr, tpr, thresholds = roc_curve(y_test, res)
    auc = roc_auc_score(y_test, res)

    fpr2, tpr2, thresholds2 = det_curve(y_test, res)

    return fpr, tpr, auc, fpr2, tpr2

def plot_roc_det_curves(name, model, ax_roc : plt.Axes, ax_det : plt.Axes, samples, modality):

    fprs = []
    tprs = []
    fprs2 = []
    tprs2 = []
    roc_aucs = []

    # fit the model for each training sample and average the fpr, tpr and roc_auc
    for f_X, f_y, f_X_2, f_y_2 in samples:

        X_train = np.load("./sampled/" + f_X)
        y_train = np.load("./sampled/" + f_y)

        X_test = np.load("./sampled/" + f_X_2)
        y_test = np.load("./sampled/" + f_y_2)

        if modality=="face":
            features_face_not_filtered = X_train[:, 9 + config.NUM_POINTS_LBP:]
            X_train, y_train = filter_faces(features_face_not_filtered, y_train)
            features_face_not_filtered = X_test[:, 9 + config.NUM_POINTS_LBP:]
            X_test, y_test = filter_faces(features_face_not_filtered, y_test)
        elif modality=="skel":
            X_train = X_train[:, 0:9]
            X_test = X_test[:, 0:9]
        elif modality=="clothes":
            X_train =  X_train[:, 9:9 + config.NUM_POINTS_LBP]
            X_test = X_test[:, 9:9+config.NUM_POINTS_LBP]
        else:
            raise Exception("Modality not supported")

        fpr, tpr, roc_auc, fpr2, tpr2 = fit_dataset(model, X_train, y_train, X_test, y_test)
        fprs.append(fpr)
        tprs.append(tpr)
        fprs2.append(fpr2)
        tprs2.append(tpr2)
        roc_aucs.append(roc_auc)

    longest_fpr = len(max(fprs, key=len))
    longest_tpr = len(max(tprs, key=len))
    longest_fpr2 = len(max(fprs2, key=len))
    longest_tpr2 = len(max(tprs2, key=len))

    # pad the shorter arrays with the last value
    for i in range(0, len(fprs)):
        fprs[i] = np.pad(fprs[i], (0, longest_fpr - len(fprs[i])), 'edge')
        tprs[i] = np.pad(tprs[i], (0, longest_tpr - len(tprs[i])), 'edge')
    
    for i in range(0, len(fprs2)):
        fprs2[i] = np.pad(fprs2[i], (0, longest_fpr2 - len(fprs2[i])), 'edge')
        tprs2[i] = np.pad(tprs2[i], (0, longest_tpr2 - len(tprs2[i])), 'edge')
    
    # average the fpr and tpr and calculate the auc
    fpr = np.mean(fprs, axis=0)
    tpr = np.mean(tprs, axis=0)
    auc = np.mean(roc_aucs)
    

    fpr2 = np.mean(fprs2, axis=0)
    tpr2 = np.mean(tprs2, axis=0)

    ax_roc.plot(fpr, tpr, label=name + " (AUC = " + str(round(auc, 2)) + ")")
    ax_det.plot(fpr2, tpr2, label=name)

    ax_roc.set_xlabel("False Positive Rate")
    ax_roc.set_ylabel("True Positive Rate")
    ax_roc.set_title(f"ROC Curve - {modality}")
    ax_det.set_xlabel("Detection Error Rate")
    ax_det.set_ylabel("Detection Rate")
    ax_det.set_title(f"DET Curve - {modality}")
    ax_roc.legend(loc="lower right")
    ax_det.legend(loc="upper right")
    ax_roc.grid(linestyle="--")
    ax_det.grid(linestyle="--")


for name, model in classifiers_skel.items():

    plot_roc_det_curves(name, model, ax_roc_skel, ax_det_skel, samples, 'skel')

for name, model in classifiers_clothes.items():
    
    plot_roc_det_curves(name, model, ax_roc_clothes, ax_det_clothes, samples, "clothes")   

for name, model in classifiers_face.items():
        
    plot_roc_det_curves(name, model, ax_roc_face, ax_det_face, samples, "face") 

plt.legend()
plt.savefig("ROC_DET_curves.png")

## Model Tuning

### Fit and Evaluate Models

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.model_selection import GridSearchCV


class_names = ['Different', 'Same']

def fit_models(models, X_train, y_train):
    for name, model in models.items():
        model.fit(X_train, y_train.ravel())
        # print(name, 'trained.')
    return

def evaluate_models(models,model_name,X_test, y_test,printable):
    for name, model in models.items():
        y_pred = model.predict(X_test)
        if printable:
            print(name)

        cv_accuracy = cross_val_score(model, X_test, y_test.ravel(), n_jobs=-1, scoring='accuracy')
        cv_f1_macro = cross_val_score(model, X_test, y_test.ravel(), n_jobs=-1, scoring='f1_macro')
        # print(cross_val_score(model, X_test, y_test, scoring='accuracy'))
        if printable:
            print(classification_report(y_test, y_pred,zero_division=0))
            print("Accuracy: %0.4f (+/- %0.4f)" % (cv_accuracy.mean(), cv_accuracy.std() * 2))
        # print(cross_val_score(model, X_test, y_test, scoring='f1_macro'))
            print("f1-score: %0.4f (+/- %0.4f)" % (cv_f1_macro.mean(), cv_f1_macro.std() * 2))
        if name.count("accuracy")> 0:
            cm = confusion_matrix(y_test, y_pred)
            cmn = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        if printable:
            print('------------------------------------')
    return cmn

    

### Model Tuning

In [ ]:
from prettytable import PrettyTable

best_param_table = PrettyTable()
def tune_model(part,model, param_grid, scoring, x_train, y_train, grid_jobs):
    # print('tuning...')
    clf = GridSearchCV(estimator=model, param_grid=param_grid, scoring=scoring, verbose=0, n_jobs=grid_jobs)
    clf.fit(x_train, y_train.ravel())
    # print('done')
    # print("%s - %s - %s: Best parameter  %s" % (model,part, scoring,clf.best_params_))
    best_params = clf.best_params_.copy()
    row = [part,scoring] + list(best_params.values()) + list([round(clf.best_score_ ,2)])
    best_param_table.add_row(row)
    return best_params




### Models

In [ ]:

SVM_param_grid = {
    'C': [0.5, 1],
    'degree': [1,2,3],
    'gamma': ['scale'],
    'kernel': ['poly', 'rbf', 'sigmoid', 'linear'],
    'random_state': [2,42]
}

RF_param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None],
    'max_features': ['sqrt', 'log2'],
    'min_samples_leaf': [1, 2, 4],
    'min_samples_split': [2, 5, 10],
    'n_estimators': [50, 100, 200],
    'random_state': [2,42]
}


Gauss_param_grid = {
    'var_smoothing': [1e-9, 1e-8, 1e-7]
}

KNN_param_grid = {
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'leaf_size': [10, 20, 30, 40],
    'n_neighbors': [3, 5, 7, 9],
    'p': [1, 2],
    'weights': ['uniform', 'distance'],

}

DT_param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None],
    'max_features': ['sqrt', 'log2'],
    'min_samples_leaf': [1, 2, 4],
    'min_samples_split': [2, 5, 10],
    'random_state': [2,42],
    'splitter': ['best', 'random']
}

Regres_param_grid = {
    'C': [0.5,1,4],
    'max_iter': [10000],
    'random_state': [2,42],
    'solver': ['lbfgs', 'liblinear', 'sag', 'saga']

}

models = {
    "KNN": (KNeighborsClassifier, KNN_param_grid),
    "DT": (DecisionTreeClassifier, DT_param_grid),
    "RF": (RandomForestClassifier, RF_param_grid),
    "GNB": (GaussianNB, Gauss_param_grid),
    "SVM": (SVC, SVM_param_grid),
    "LogReg": (LogisticRegression, Regres_param_grid),
}

### Tuning and Evaluation

#### Train

In [ ]:
confusion_matrixes_train = {"Skel": dict(), "Clothes":dict() , "Face":dict() }


for f_X, f_y, f_X_2, f_y_2 in samples:

    
    X = np.load(config.SAMPLED_PATH + f_X)
    y = np.load(config.SAMPLED_PATH + f_y)
    X_skel, X_clothes, X_face, _, ___, ____, y_train, _____, y_face, ______ = filter_and_split_dataset(X, y)

    X_2 = np.load(config.SAMPLED_PATH + f_X_2)
    y_2 = np.load(config.SAMPLED_PATH + f_y_2)
    X_skel_test, X_clothes_test, X_face_test, _, __, ___, y_test, ____, y_face_test, _____ = filter_and_split_dataset(X_2, y_2)


    for name, model in models.items():
        printable = False
        if name not in confusion_matrixes_train["Skel"]:
            confusion_matrixes_train["Skel"][name] = []
            confusion_matrixes_train["Clothes"][name] = []
            confusion_matrixes_train["Face"][name] = []
        if printable:
            print(name)
        current_model = model
        best_param_table = PrettyTable()
        best_param_table.title = model
        best_param_table.field_names = [" ","scoring"] + list(current_model[1].keys()) + ["score"]
        if printable:
            print("Tuning...")
        best_accuracy_params_skel = tune_model("Skeleton",current_model[0](), current_model[1], 'accuracy', X_skel, y_train, -1)
        best_f1macro_params_skel = tune_model("Skeleton",current_model[0](), current_model[1], 'f1_macro',  X_skel, y_train, -1)
        best_accuracy_params_clothes = tune_model("Clothes",current_model[0](), current_model[1], 'accuracy', X_clothes, y_train, -1)
        best_f1macro_params_clothes = tune_model("Clothes",current_model[0](), current_model[1], 'f1_macro',  X_clothes, y_train, -1)
        best_accuracy_params_face = tune_model("Face",current_model[0](), current_model[1], 'accuracy', X_face, y_face, -1)
        best_f1macro_params_face = tune_model("Face",current_model[0](), current_model[1], 'f1_macro',  X_face, y_face, -1)
        if f_X == "X_1.npy" and f_y == "y_1.npy":
            printable = True
            print(best_param_table)
        tuned_models_skel = {
            'Skeleton accuracy': current_model[0](**best_accuracy_params_skel),
            'Skeleton f1_macro': current_model[0](**best_f1macro_params_skel)
        }

        tuned_models_clothes = {
            'Clothes accuracy': current_model[0](**best_accuracy_params_clothes),
            'Clothes f1_macro': current_model[0](**best_f1macro_params_clothes)
        }

        tuned_models_face = {
            'Face accuracy': current_model[0](**best_accuracy_params_face),
            'Face f1_macro': current_model[0](**best_f1macro_params_face)
        }
        fit_models(tuned_models_skel, X_skel, y_train)
        fit_models(tuned_models_clothes, X_clothes, y_train)
        fit_models(tuned_models_face, X_face, y_face)
        if printable:
            print("-"*50)
            print("Skeleton")
        confusion_matrixes_train["Skel"][name].append(evaluate_models(tuned_models_skel,model, X_skel, y_train,printable))
        if printable:
            print("-"*50)
            print("Clothes")
            print("-"*50)
        confusion_matrixes_train["Clothes"][name].append(evaluate_models(tuned_models_clothes,model, X_clothes, y_train,printable))
        if printable:
            print("-"*50)
            print("Face")
            print("-"*50)
        confusion_matrixes_train["Face"][name].append(evaluate_models(tuned_models_face,model, X_face, y_face,printable))



##### Confusion Matrix

In [ ]:
cols = ['{}'.format(col) for col in ["Skeleton", "Clothes", "Face"]]
rows = ['{}'.format(row) for row in models.keys()]


fig, axes = plt.subplots(nrows=len(models), ncols=3, figsize=(15,15))
for i in range(len(models)):
    name = list(models.keys())[i]

    d1 = ConfusionMatrixDisplay(np.mean(confusion_matrixes_train["Skel"][name],axis=0),display_labels= class_names)
    d1.plot(ax=axes[i,0], cmap=plt.cm.Purples)
    d2 = ConfusionMatrixDisplay(np.mean(confusion_matrixes_train["Clothes"][name],axis=0),display_labels= class_names)
    d2.plot(ax=axes[i,1], cmap=plt.cm.Purples)
    d3 = ConfusionMatrixDisplay(np.mean(confusion_matrixes_train["Face"][name],axis=0),display_labels= class_names)
    d3.plot(ax=axes[i,2], cmap=plt.cm.Purples)
    
for ax, col in zip(axes[0], cols):
    ax.set_title(col)

for ax, row in zip(axes[:,0], rows):
    ax.set_ylabel(row, rotation=0, size='large')

fig.suptitle('Confusion Matrixes Train', fontsize=16)

fig.tight_layout()  
plt.show()
fig.savefig('confusion_matrixes_train.png')

### Test

In [ ]:
confusion_matrixes_test = {"Skel": dict(), "Clothes":dict() , "Face":dict() }


for f_X, f_y, f_X_2, f_y_2 in samples:

    printable = False
    X = np.load(config.SAMPLED_PATH + f_X)
    y = np.load(config.SAMPLED_PATH + f_y)
    X_skel, X_clothes, X_face, _, ___, ____, y_train, _____, y_face, ______ = filter_and_split_dataset(X, y)

    X_2 = np.load(config.SAMPLED_PATH + f_X_2)
    y_2 = np.load(config.SAMPLED_PATH + f_y_2)
    X_skel_test, X_clothes_test, X_face_test, _, __, ___, y_test, ____, y_face_test, _____ = filter_and_split_dataset(X_2, y_2)


    for name, model in models.items():
        if name not in confusion_matrixes_test["Skel"]:
            confusion_matrixes_test["Skel"][name] = []
            confusion_matrixes_test["Clothes"][name] = []
            confusion_matrixes_test["Face"][name] = []
        print(name)
        current_model = model
        best_param_table = PrettyTable()
        best_param_table.title = model
        best_param_table.field_names = [" ","scoring"] + list(current_model[1].keys()) + ["score"]
        print("Tuning...")
        best_accuracy_params_skel = tune_model("Skeleton",current_model[0](), current_model[1], 'accuracy', X_skel, y_train, -1)
        best_f1macro_params_skel = tune_model("Skeleton",current_model[0](), current_model[1], 'f1_macro',  X_skel, y_train, -1)
        best_accuracy_params_clothes = tune_model("Clothes",current_model[0](), current_model[1], 'accuracy', X_clothes, y_train, -1)
        best_f1macro_params_clothes = tune_model("Clothes",current_model[0](), current_model[1], 'f1_macro',  X_clothes, y_train, -1)
        best_accuracy_params_face = tune_model("Face",current_model[0](), current_model[1], 'accuracy', X_face, y_face, -1)
        best_f1macro_params_face = tune_model("Face",current_model[0](), current_model[1], 'f1_macro',  X_face, y_face, -1)
        if f_X == "X_1.npy" and f_y == "y_1.npy":
            print(best_param_table)
            printable = True
        tuned_models_skel = {
            'Skeleton accuracy': current_model[0](**best_accuracy_params_skel),
            'Skeleton f1_macro': current_model[0](**best_f1macro_params_skel)
        }

        tuned_models_clothes = {
            'Clothes accuracy': current_model[0](**best_accuracy_params_clothes),
            'Clothes f1_macro': current_model[0](**best_f1macro_params_clothes)
        }

        tuned_models_face = {
            'Face accuracy': current_model[0](**best_accuracy_params_face),
            'Face f1_macro': current_model[0](**best_f1macro_params_face)
        }
        fit_models(tuned_models_skel, X_skel, y_train)
        fit_models(tuned_models_clothes, X_clothes, y_train)
        fit_models(tuned_models_face, X_face, y_face)
        if printable:
            print("-"*50)
            print("Skeleton")
        confusion_matrixes_test["Skel"][name].append(evaluate_models(tuned_models_skel,model, X_skel_test, y_test,printable))
        if printable:
            print("-"*50)
            print("Clothes")
            print("-"*50)
        confusion_matrixes_test["Clothes"][name].append(evaluate_models(tuned_models_clothes,model, X_clothes_test, y_test,printable))
        if printable:
            print("-"*50)
            print("Face")
            print("-"*50)
        confusion_matrixes_test["Face"][name].append(evaluate_models(tuned_models_face,model, X_face_test, y_face_test,printable))



##### Confusion Matrix

In [ ]:
cols = ['{}'.format(col) for col in ["Skeleton", "Clothes", "Face"]]
rows = ['{}'.format(row) for row in models.keys()]


fig, axes = plt.subplots(nrows=len(models), ncols=3, figsize=(15,15))
for i in range(len(models)):
    name = list(models.keys())[i]

    d1 = ConfusionMatrixDisplay(np.mean(confusion_matrixes_test["Skel"][name],axis=0),display_labels= class_names)
    d1.plot(ax=axes[i,0], cmap=plt.cm.Purples)
    d2 = ConfusionMatrixDisplay(np.mean(confusion_matrixes_test["Clothes"][name],axis=0),display_labels= class_names)
    d2.plot(ax=axes[i,1], cmap=plt.cm.Purples)
    d3 = ConfusionMatrixDisplay(np.mean(confusion_matrixes_test["Face"][name],axis=0),display_labels= class_names)
    d3.plot(ax=axes[i,2], cmap=plt.cm.Purples)
    
for ax, col in zip(axes[0], cols):
    ax.set_title(col)

for ax, row in zip(axes[:,0], rows):
    ax.set_ylabel(row, rotation=0, size='large')

fig.suptitle('Confusion Matrixes Test', fontsize=16)

fig.tight_layout()  
plt.show()
fig.savefig('confusion_matrixes_test.png')

# Final Model

The following block contains the final model and the tests.

In [ ]:
class FinalModel(BaseEstimator, ClassifierMixin):

    def __init__(self, policy='argmax', name = "FinalModel"):
        self.model_skel = RandomForestClassifier(criterion ='entropy', max_depth = None, max_features ='sqrt', min_samples_leaf = 1, min_samples_split = 2, n_estimators = 200, random_state = 42)
        self.model_clothes = RandomForestClassifier(criterion ='entropy', max_depth = None, max_features ='sqrt', min_samples_leaf = 1, min_samples_split = 10, n_estimators = 100, random_state = 4)
        self.model_face = SVC(C = 1, kernel = "linear", degree = 3, gamma = "auto", probability = True, tol = 0.001, random_state = 42)
        self.name = name
        self.classes_ = [0, 1]
        self.policy = policy

    def fit(self, X, y):
        # prepare data
        features_skel = X[:, 0:9]
        features_clothes =  X[:, 9:9 + config.NUM_POINTS_LBP]
        features_face_not_filtered = X[:, 9 + config.NUM_POINTS_LBP:]
        features_face, y_face = filter_faces(features_face_not_filtered, y)

        # fit models
        self.model_skel.fit(features_skel, y)
        self.model_clothes.fit(features_clothes, y)
        self.model_face.fit(features_face, y_face)
        return self

    def predict(self, X):
        if self.policy == 'argmax':
            return np.argmax(self.predict_proba(X), axis=1)
        elif self.policy == 'or':
            features_skel = X[:, 0:9]
            features_clothes =  X[:, 9:9 + config.NUM_POINTS_LBP]
            features_face_not_filtered = X[:, 9 + config.NUM_POINTS_LBP:]
            
            indexes = np.argwhere(~np.isnan(features_face_not_filtered[:, 0].reshape(-1)))
            features_face = features_face_not_filtered[indexes.flatten()]
            
            output_skel = self.model_skel.predict(features_skel)
            output_clothes = self.model_clothes.predict(features_clothes)
            o_face = self.model_face.predict(features_face)

            output_face = np.zeros(len(X))
            np.put(output_face, indexes, o_face)

            return np.bitwise_or(output_skel, output_clothes, output_face)
        else:
            raise Exception(f"Policy not supported: {self.policy}. Please use \'or\' | \'argmax\'")

    def predict_proba(self, X):
        features_skel = X[:, 0:9]
        features_clothes =  X[:, 9:9 + config.NUM_POINTS_LBP]
        features_face_not_filtered = X[:, 9 + config.NUM_POINTS_LBP:]

        # Get valid faces
        indexes = np.argwhere(~np.isnan(features_face_not_filtered[:, 0].reshape(-1)))
        features_face = features_face_not_filtered[indexes.flatten()]
        
        # Predict
        output_skel = self.model_skel.predict_proba(features_skel)
        output_clothes = self.model_clothes.predict_proba(features_clothes)
        o_face = self.model_face.predict_proba(features_face)

        # Insert face outputs in a new array
        output_face = np.asarray([np.asarray([0, 0]) for _ in range(len(X))])
        np.put(output_face, indexes, o_face)

        # calculate final probabilities
        output = (output_skel + output_clothes + output_face) / 3
        
        return output

In [ ]:

files = [f for f in os.listdir("./sampled") if re.match("(video_)([0-9]*)(_sample_)([0-9]*)_([Xy])(.npy)", f)]
files.sort()
samples = [(files[i], files[i+1], files[i+2], files[i+3]) for i in range(0, len(files) - 2, 4)]

fig, (ax_roc, ax_det) = plt.subplots(1, 2)
fig.set_size_inches(14, 6)

FINAL_MODEL_POLICY = "argmax"

fprs = []
tprs = []
fprs2 = []
tprs2 = []
roc_aucs = []
cms = []
# fit the model for each training sample and average the fpr, tpr and roc_auc
for f_X, f_y, f_X_2, f_y_2 in samples:
    print([f_X, f_y, f_X_2, f_y_2])

    X = np.load("./sampled/" + f_X)
    y = np.load("./sampled/" + f_y)

    X_2 = np.load("./sampled/" + f_X_2)
    y_2 = np.load("./sampled/" + f_y_2)

    model = FinalModel(policy = FINAL_MODEL_POLICY)
    model.fit(X, y)

    res = model.predict_proba(X_2)[:, 1]

    pr = model.predict(X_2)

    print(classification_report(y_2, pr))

    cms.append(confusion_matrix(y_2, pr))
    
    fpr, tpr, thresholds = roc_curve(y_2, res)
    auc = roc_auc_score(y_2, res)
    roc_aucs.append(auc)

    fprs.append(fpr)
    tprs.append(tpr)

    fpr2, tpr2, thresholds2 = det_curve(y_2, res)
    fprs2.append(fpr2)
    tprs2.append(tpr2)

longest_fpr = len(max(fprs, key=len))
longest_tpr = len(max(tprs, key=len))
longest_fpr2 = len(max(fprs2, key=len))
longest_tpr2 = len(max(tprs2, key=len))

# pad the shorter arrays with the last value
for i in range(0, len(fprs)):
    fprs[i] = np.pad(fprs[i], (0, longest_fpr - len(fprs[i])), 'edge')
    tprs[i] = np.pad(tprs[i], (0, longest_tpr - len(tprs[i])), 'edge')

for i in range(0, len(fprs2)):
    fprs2[i] = np.pad(fprs2[i], (0, longest_fpr2 - len(fprs2[i])), 'edge')
    tprs2[i] = np.pad(tprs2[i], (0, longest_tpr2 - len(tprs2[i])), 'edge')

# average the fpr and tpr and calculate the auc
fpr = np.mean(fprs, axis=0)
tpr = np.mean(tprs, axis=0)
fpr2 = np.mean(fprs2, axis=0)
tpr2 = np.mean(tprs2, axis=0)

auc = np.mean(roc_aucs)

cms = np.asarray(cms)
print(cms.shape)
cm = np.mean(cms, axis=0)

fig_cm, ax_cm = plt.subplots(1, 1)
ax_cm.set_title(f"FinalModel - {FINAL_MODEL_POLICY}")
disp = ConfusionMatrixDisplay(cm)
ax_cm = disp.plot(cmap=plt.cm.Purples, ax=ax_cm)
plt.savefig(f"final_model_{FINAL_MODEL_POLICY}_cm.png")

ax_roc.set_title("FinalModel - ROC")
ax_det.set_title("FinalModel - DET")


ax_roc.set_xlabel("False Positive Rate")
ax_det.set_xlabel("Detection Error Rate")

ax_roc.set_ylabel("True Positive Rate")
ax_det.set_ylabel("Detection Rate")

# Add Legends

ax_roc.grid(linestyle="--")
ax_det.grid(linestyle="--")


ax_roc.plot(fpr, tpr, label="FinalModel (AUC = " + str(round(auc, 2)) + ")")
ax_det.plot(fpr2, tpr2, label="FinalModel")
ax_roc.legend(loc="lower right")
ax_det.legend(loc="upper right")
fig.savefig(f"final_model_roc_det.png")

## Regression

We also thought that it was possible to learn a "latent space" that ensambles that person's features. The idea is that a regressor can learn how to optimize the outpot of a linear combination so that features from the person that we want to re-identify will output a low value (0) while the other people will output an higher value (1).

Given that, we used the AdaBoost with SVR regressors in a pipeline in order to learn the features of a person.

In [ ]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.metrics import label_ranking_average_precision_score
    
def test_model_(name, model, X_train, X_test, y_train, y_test, ax):
    # fit the model on the training set
    model.fit(X_train, y_train)
    # Test
    pred = model.predict(X_test)

    rerank = [np.asarray([a, b]) for a, b in zip(pred, y_test)]
    rerank.sort(key=lambda x: x[0])

    other = target = 0

    l = []
    for idx, element in enumerate(rerank):
        if element[1] == 0:
            target += 1
        else:
            other += 1
        l.append(target)
                
    # Plots

    if ax is not None: 
        print("-" * 10)
        print("|", name, "|")
        print("-" * 10)    
        pca = PCA(n_components=2)
        pca_result = pca.fit_transform(X_train)
        print('Explained variation per principal component: {}'.format(pca.explained_variance_ratio_))

        plt.figure(figsize=(10,6))
        plt.scatter(
            pca_result[:, 0], pca_result[:, 1],
            c=y_train,
            cmap='prism'
        )
        plt.xlabel('Principal Component 1')
        plt.ylabel('Principal Component 2')
        plt.show()

        # tsne = TSNE(n_components=2, verbose=1, perplexity=100, n_iter=1000)
        # tsne_results = tsne.fit_transform(X_train, y=y_train)

        # plt.figure(figsize=(10,6))
        # plt.scatter(
        #     tsne_results[:, 0], tsne_results[:, 1],
        #     c=y_train,
        #     cmap='prism'
        # )
        # plt.xlabel('t-SNE Component 1')
        # plt.ylabel('t-SNE Component 2')
        # plt.show()
    
        print(mean_absolute_error(y_test, pred))
        print(mean_squared_error(y_test, pred))
        plt.plot(y_test, pred, 'o')    
        plt.show()
    l = np.asarray(l)
    return l / (target)

plot the number of true positive (normalized) as long as we consider samples of the re ranked input.

In [ ]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn.linear_model import BayesianRidge

files = [f for f in os.listdir("./sampled") if re.match("(video_)([0-9]*)(_sample_)([0-9]*)_([Xy])(.npy)", f)]
files.sort()
samples = [(files[i], files[i+1], files[i+2], files[i+3]) for i in range(0, len(files) - 2, 4)]

curve_sk = []
curve_cl = []
curve_fa = []

for f_X, f_y, f_X_2, f_y_2 in samples:
    print([f_X, f_y, f_X_2, f_y_2])

    X = np.load("./sampled/" + f_X)
    y = np.load("./sampled/" + f_y)
    y -= 1
    y *= -1
    X_skel, X_clothes, X_face, trhs_skel, trhs_clothes, trhs_faces, y_train, y_trhs, y_face, y_trhs_face = filter_and_split_dataset(X, y)

    X_2 = np.load("./sampled/" + f_X_2)
    y_2 = np.load("./sampled/" + f_y_2)
    y_2 -= 1
    y_2 *= -1
    X_skel_test, X_clothes_test, X_face_test, _, __, ___, y_test, ____, y_face_test, _____ = filter_and_split_dataset(X_2, y_2)


    m1 = Pipeline([("poly", PolynomialFeatures()), ("scaler", StandardScaler()), ("SVRLogistic", AdaBoostRegressor(base_estimator=SVR(), n_estimators=30))])
    m2 = Pipeline([("poly", PolynomialFeatures()), ("scaler", StandardScaler()), ("SVRLogistic", AdaBoostRegressor(base_estimator=SVR(), n_estimators=30))])
    m3 = Pipeline([("poly", PolynomialFeatures()), ("scaler", StandardScaler()), ("SVRLogistic", SVR())])
    # fig, ax = plt.subplots(1, 3)
    ax = [None] * 3
    curve_sk.append(test_model_("Skel", m1, X_skel, X_skel_test, y_train, y_test, ax[0]))
    curve_cl.append(test_model_("Clothes", m2, X_clothes, X_clothes_test, y_train, y_test, ax[1]))
    curve_fa.append(test_model_("Face", m3, X_face, X_face_test, y_face, y_face_test, ax[2]))

curve_sk = np.mean(curve_sk, axis=0)
curve_cl = np.mean(curve_cl, axis=0)
x_range = np.arange(0, len(curve_sk), dtype=np.float32) / len(curve_sk)
plt.plot(x_range, curve_sk, label="skeleton")
plt.plot(x_range, curve_cl, label="clothes")

# padding

longest_fa = len(max(curve_fa, key=len))

# pad the shorter arrays with the last value
for i in range(0, len(curve_fa)):
    curve_fa[i] = np.pad(curve_fa[i], (0, longest_fa - len(curve_fa[i])), 'edge')

curve_fa = np.mean(curve_fa, axis=0)
x_range_fa = np.arange(0, len(curve_fa), dtype=np.float32)
plt.plot(x_range_fa / len(curve_fa), curve_fa, label="face")
plt.title("Regression - ranking")
plt.xlabel("Number of samples (normalized)")
plt.ylabel("number of true labels (normalized)")

# Add Legends

plt.grid(linestyle="--")
plt.legend()
plt.show()

### FinalRegressor

final regressor made by the sum of three minor regressors.

In [ ]:
class FinalRegressor(BaseEstimator, RegressorMixin):
    def __init__(self):
        """
        Called when initializing the classifier
        """
        self.model_skel = Pipeline([("poly", PolynomialFeatures()), ("scaler", StandardScaler()), ("SVRLogistic", AdaBoostRegressor(base_estimator=SVR(), n_estimators=30))])
        self.model_clothes = Pipeline([("poly", PolynomialFeatures()), ("scaler", StandardScaler()), ("SVRLogistic", AdaBoostRegressor(base_estimator=SVR(), n_estimators=30))])
        self.model_face = Pipeline([("poly", PolynomialFeatures()), ("scaler", StandardScaler()), ("SVRLogistic", SVR())])

    def fit(self, X, y=None):        
        features_skel = X[:, 0:9]
        features_clothes =  X[:, 9:9 + config.NUM_POINTS_LBP]
        features_face_not_filtered = X[:, 9 + config.NUM_POINTS_LBP:]
        features_face, y_face = filter_faces(features_face_not_filtered, y)

        # fit models
        self.model_skel.fit(features_skel, y)
        self.model_clothes.fit(features_clothes, y)
        self.model_face.fit(features_face, y_face)
        return self

    def predict(self, X, y=None):
        features_skel = X[:, 0:9]
        features_clothes =  X[:, 9:9 + config.NUM_POINTS_LBP]
        features_face_not_filtered = X[:, 9 + config.NUM_POINTS_LBP:]
            
        indexes = np.argwhere(~np.isnan(features_face_not_filtered[:, 0].reshape(-1)))
        features_face = features_face_not_filtered[indexes.flatten()]
        
        output_skel = self.model_skel.predict(features_skel)
        output_clothes = self.model_clothes.predict(features_clothes)
        o_face = self.model_face.predict(features_face)

        output_face = np.ones(len(X))
        np.put(output_face, indexes, o_face)

        return (output_skel + output_clothes + output_face) / 3

In [ ]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn.linear_model import BayesianRidge

files = [f for f in os.listdir("./sampled") if re.match("(video_)([0-9]*)(_sample_)([0-9]*)_([Xy])(.npy)", f)]
files.sort()
samples = [(files[i], files[i+1], files[i+2], files[i+3]) for i in range(0, len(files) - 2, 4)]

curve_fr = []

for f_X, f_y, f_X_2, f_y_2 in samples:
    print([f_X, f_y, f_X_2, f_y_2])

    X = np.load("./sampled/" + f_X)
    y = np.load("./sampled/" + f_y)
    y -= 1
    y *= -1

    X_2 = np.load("./sampled/" + f_X_2)
    y_2 = np.load("./sampled/" + f_y_2)
    y_2 -= 1
    y_2 *= -1

    m = FinalRegressor()
    # fig, ax = plt.subplots(1, 3)
    ax = [None] * 3
    curve_fr.append(test_model_("Skel", m, X, X_2, y, y_2, ax[0]))


curve_fr = np.mean(curve_fr, axis=0)
x_range = np.arange(0, len(curve_fr), dtype=np.float32) / len(curve_fr)
plt.plot(x_range, curve_fr, label="FinalRegressor")

# padding

plt.title("FinalRegressor - ranking")
plt.xlabel("Number of samples (normalized)")
plt.ylabel("number of true labels (normalized)")

# Add Legends

plt.grid(linestyle="--")
plt.legend()
plt.show()